In [1]:
# -*- coding: utf-8 -*- 

import re 
from tqdm import tqdm 
import time
from datetime import datetime
import sqlite3
import sys
import os
import pandas as pd
import unify

In [2]:
def connect(file_path, primary, columns):
    con = sqlite3.connect(file_path)
    cur = con.cursor()
    cols = ", ".join([c + ' Varchar' for c in columns]) 
    cur.execute("create table meta ("+primary+" Varchar PRIMARY KEY, "+cols+" )")
    cur.execute("CREATE INDEX log on meta (textid);")
    cur.execute("create table plain_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL);")
    cur.execute("create table tagged_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL );")
    con.commit()
    return con, cur

In [3]:
workdir = r'/home/tari/Загрузки/taiga/proza_ru'
filename = 'proza_ru.db'
file_path = os.path.join(workdir, filename)
metatablepath = os.path.join(workdir,'newmetadata.csv')
tagged = os.path.join(workdir,'texts_tagged')
plain = os.path.join(workdir,'texts')

meta = pd.read_csv(metatablepath, sep='\t', encoding='utf8')
meta = meta.fillna('')
meta.head()

,segment,textid,textname,textregion,textrubric,textdiff,author,authortexts,authorreaders,magazine,date,time,tags,source
0,proza_ru,201612211706,Либеральный патриот VS патриотичный либерал,,публицистика,,Михаил Хаймович,22,7368,,21.12.2016,19:24,,http://www.proza.ru/2016/12/21/1706
1,proza_ru,201510251942,Яблочный спас.,,миниатюры,,Михаил Хаймович,22,7368,,25.10.2015,21:44,,http://www.proza.ru/2015/10/25/1942
2,proza_ru,201510171907,Дядя Дима - полковник.,,рассказы,,Михаил Хаймович,22,7368,,17.10.2015,22:04,,http://www.proza.ru/2015/10/17/1907
3,proza_ru,201508081131,Мы - офицеры!,,рассказы,,Михаил Хаймович,22,7368,,08.08.2015,15:33,,http://www.proza.ru/2015/08/08/1131
4,proza_ru,201404032273,Кольцо,,миниатюры,,Михаил Хаймович,22,7368,,03.04.2014,22:43,,http://www.proza.ru/2014/04/03/2273


In [4]:
if not os.path.exists(filename):
    con, cur = connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
else:
    con = sqlite3.connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
    cur = con.cursor()

In [5]:
meta.iloc[6].to_dict()

{'author': 'Михаил Хаймович',
 'authorreaders': 7368,
 'authortexts': 22,
 'date': '12.12.2012',
 'magazine': '',
 'segment': 'proza_ru',
 'source': 'http://www.proza.ru/2012/12/12/1180',
 'tags': '',
 'textdiff': '',
 'textid': '201212121180',
 'textname': 'На переломе',
 'textregion': '',
 'textrubric': 'рассказы',
 'time': '14:41'}

In [ ]:
for i in range(len(meta)):
    values = meta.iloc[i].to_dict()
    values['textid'] = str(values['textid'])
    values['textdiff'] = str(values['textdiff'])
    values['authorreaders'] = str(values['authorreaders'])
    values['authortexts'] = str(values['authortexts'])
    
    
    columns = ', '.join(values.keys())
    
    #print(list(values.values()))
    placeholders = ', '.join('?' * len(values))
    sql = 'INSERT INTO meta ({}) VALUES ({})'.format(columns, placeholders)
    #print(sql)
    cur.execute(sql, list(values.values()))
    
    valuest = {'id': values['textid'], 'text': unify.open_text(os.path.join(plain, str(values['textid'])+".txt"))}
    
    columns = ', '.join(valuest.keys())
    placeholders = ', '.join('?' * len(valuest))
    sql2 = 'INSERT INTO plain_texts ({}) VALUES ({})'.format(columns, placeholders) 
    cur.execute(sql2, list(valuest.values()))
    try:
        valuest2 = {'id': values['textid'], 'text': unify.open_text(open(os.path.join(tagged, str(values['textid'])+".txt"),'r', encoding='utf8').read())}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    except:
        valuest2 = {'id': values['textid'], 'text': ""}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    con.commit()